# Sequence-to-sequence

본 ipython notebook은 [DIYA](https://blog.diyaml.com/) 회원들의 자연어처리 스터디를 위해, 아래의 자료를 바탕으로 만들어졌습니다.
* [Stanford CS224N Assignment 4](http://web.stanford.edu/class/cs224n/assignments/a4.pdf)
* [PyTorch Seq2Seq](https://github.com/bentrevett/pytorch-seq2seq)
* [네이버 리뷰 데이터를 활용한 한글 데이터 감정 분석](https://github.com/reniew/NSMC_Sentimental-Analysis)
* [토치텍스트 튜토리얼(Torchtext tutorial) - 한국어](https://wikidocs.net/65348)

본 실습의 구성은 다음과 같습니다.
1. [torchtext를 이용한 데이터셋 생성](#Data-Iterator)
2. [Seq2Seq 구현](#Seq2Seq-Model)
3. [Seq2Seq 학습](#Training-the-Model)

In [ ]:
"""
박규병님께서 Kaggle에 올려주신 1000sents 데이터를 다운로드합니다.
torchtext에서의 Field 기능을 활용하기 위해, 전처리는 하지 않았습니다.
"""
import requests

# Get file link from google drive
file_id = "1R_XGHvkgMArQM6SM59_U7qLaDD2gb3FZ"
file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id

# Download and unzip file
data = requests.get(file_download_link)
filename = '1000sents.csv'
with open(filename, 'wb') as f:
    f.write(data.content)

## Data Iterator

In [ ]:
"""
토큰화를 위한 형태소분석기를 정의해줍니다.
"""
!pip install konlpy -q
!pip install torchtext==0.6.0 -q

import re
from konlpy.tag import Okt

okt = Okt()  # 이번에는 과거 트위터 형태소 분석기라고 부르던 Okt 분석기를 사용해 보겠습니다.
stop_words = [  # 불용어를 정의합니다.
    '은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수',
    '보', '주', '등', '한', '을', '를'
]

def tokenize(text):
    """code modified from https://github.com/reniew/NSMC_Sentimental-Analysis"""
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text)

    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)

    # 3. 불용어 제거
    word_review = [token for token in word_review if not token in stop_words]

    return word_review

In [ ]:
"""
torchtext의 데이터셋 형태로 변환하기 위해,
데이터의 각 필드를 정의해줍니다.
"""
from torchtext import data

SRC = data.Field(       # 한국어 문장
    tokenize=tokenize,
    init_token='<sos>', # 문장의 시작 토큰
    eos_token='<eos>',  # 문장의 끝 토큰
    include_lengths=True
)
TRG = data.Field(       # 영어 문장
    tokenize='spacy',
    init_token='<sos>',
    eos_token='<eos>',
    lower=True
)

train_data = data.TabularDataset(
    path='1000sents.csv',
    format='csv',
    fields=[('korean', SRC), ('english', TRG)],
    skip_header=True
)

In [ ]:
# 샘플 데이터를 출력합니다.
for i in range(10):
    print(train_data[i].korean, '\t', train_data[i].english)

In [ ]:
"""
각기 다른 형태소들을 (distinct words) 숫자로 변환해줍니다.
최소 3번 이상 등장한 형태소들에 대해서만 분석을 수행합니다.
"""
SRC.build_vocab(train_data, min_freq=3)
TRG.build_vocab(train_data, min_freq=3)

print('단어 집합의 크기 : 한국어 {}개  영어 {}개'.format(len(SRC.vocab), len(TRG.vocab)))
print(list(SRC.vocab.stoi.items()))

In [ ]:
"""
torchtext의 BucketIterator 객체를 통해 서로 다른 길이의 문장들을 한번에 불러올 수 있습니다.
배치 내에 가장 긴 문장을 첫번째에 두고, 이후의 문장들은 뒤에 <pad> 토큰을 붙임으로써 첫번째 문장과 길이를 맞춰줍니다.
"""
import itertools

batch_size = 32
train_iterator = data.BucketIterator(
    train_data, 
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x : len(x.korean),
)

# padding이 잘 들어갔는지 확인해봅시다.
sample = next(itertools.islice(train_iterator, 4, None)).korean[0]
print(sample.shape)
print(sample.T)

## Seq2Seq Model

![Seq2Seq Overview](https://github.com/bentrevett/pytorch-seq2seq/raw/d876a1dcacd7aeeeeeaff2c9b806d23116df048f/assets/seq2seq7.png)

이 notebook에서 구현할 sequence-to-sequence 모델은 attention과 bidirectional GRU를 활용한, [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)에서 소개한 모델입니다.

아래 순서를 따라 이 모델의 구성요소들을 하나씩 구현해봅시다.  
* Encoder
    * Word Embedding (Korean)
    * Bidirectional GRU
    * Fully Connected Layer
* Attention
    * Fully Connected Layer
    * Masking
* Decoder
    * Word Embedding (English)
    * Unidirectional GRU
    * Fully Connected Layer

### Encoder

![Encoder Overview](https://github.com/bentrevett/pytorch-seq2seq/raw/d876a1dcacd7aeeeeeaff2c9b806d23116df048f/assets/seq2seq8.png)

이 모델의 Encoder는 다음의 3가지 부분으로 이루어져있습니다.  
* Word Embedding (Korean)
* Bidirectional GRU
* Fully Connected Layer

각 레이어의 형태를 아래 코드와 같이 정의할 때, Bidirectional GRU의 forward hidden state와 backward hidden state를 합쳐 (concatenate) Fully Connected Layer로 전달해주세요.


In [ ]:
"""TODO
Encoder Module의 forward 함수를 완성해주세요.
"""
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, encoder_dim, decoder_dim,
                 dropout=0.5):
        super().__init__()
        self.embed = nn.Sequential(
            nn.Embedding(input_dim, embed_dim),
            nn.Dropout(dropout)
        )
        self.rnn = nn.GRU(embed_dim, encoder_dim, bidirectional=True)
        self.fc = nn.Linear(encoder_dim * 2, decoder_dim)

    def forward(self, src, src_len):
        embed = self.embed(src)
        
        # 각 문장의 길이를 명시해줌으로써 rnn의 hidden state에 불필요한 padding이 포함되지 않도록 합니다.
        packed_embed = nn.utils.rnn.pack_padded_sequence(embed, src_len)
        packed_outputs, hidden = self.rnn(packed_embed)

        # zero-padding을 통해 rnn의 출력값을 다시 동일한 길이로 맞춰줍니다.
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)

        # TODO: hidden의 0번 dimension을 합쳐서 fully connected로 전달해줍니다.
        hidden_cat = None
        hidden = torch.tanh(self.fc(hidden_cat))
        return outputs, hidden

# Tests
encoder = Encoder(len(SRC.vocab), 2, 3, 4)
for _ in range(3):
    src, src_len = next(iter(train_iterator)).korean
    outputs, hidden = encoder(src, src_len)
    assert outputs.shape == torch.Size((max(src_len), batch_size, 6))
    assert hidden.shape == torch.Size((batch_size, 4))
print("All tests passed")

### Attention

![Attention Overview](https://github.com/bentrevett/pytorch-seq2seq/raw/d876a1dcacd7aeeeeeaff2c9b806d23116df048f/assets/seq2seq9.png)

encoder output을 $H$, 이전 decoder hidden state을 $s_{t-1}$라 할 때 이 모델에서의 attention $a_t$은 다음과 같이 주어집니다.

$$
\begin{align}
E_t &= (\text{tanh} \circ \text{fc})\left(s_{t-1} \oplus H\right) \\
\hat{a_t} &= v^T E_t \\
a_t &= \text{softmax}(\hat{a_t})
\end{align}
$$

이때 $\oplus$는 두 벡터의 결합연산(concatenate)을 나타내고, $v$는 $E_t$와 크기가 같은 벡터로 모델이 경사하강을 통해 학습하는 모수입니다.

위 수식을 보고 아래의 Attention Module을 구현해주세요.

In [ ]:
"""TODO
Attention Module의 forward 함수를 완성해주세요.
"""

class Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim):
        super().__init__()
        self.fc = nn.Linear(encoder_dim * 2 + decoder_dim, decoder_dim)
        self.v = nn.Linear(decoder_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs, mask):
        # hidden: [batch_size, decoder_dim]
        # encoder_outputs: [max(src_len), batch_size, encoder_dim * 2]
        
        # TODO: hidden_cat의 크기가 다음과 같이 되도록 두 텐서를 재배열해주세요.
        # hidden_cat: [batch_size, max(src_len), encoder_dim * 2 + decoder_dim]
        hidden_cat = None

        energy = torch.tanh(self.fc(hidden_cat))
        attention = self.v(energy).squeeze(-1)

        # mask가 False인 곳(padding)은 전부 -inf로 채웁니다.
        attention = attention.masked_fill(~mask, -1e10)  
        
        return torch.softmax(attention, dim=-1)

# Tests
encoder = Encoder(len(SRC.vocab), 2, 3, 4)
attention = Attention(3, 4)
pad_idx = SRC.vocab.stoi[SRC.pad_token]
for _ in range(3):
    src, src_len = next(iter(train_iterator)).korean
    outputs, hidden = encoder(src, src_len)
    mask = (src != pad_idx).T  # padding이 아닌 곳에만 attention을 계산합니다.
    a = attention(hidden, outputs, mask)
    assert a.shape == torch.Size((outputs.size(1), max(src_len)))
print("All tests passed")

### Decoder

![Decoder Overview](https://github.com/bentrevett/pytorch-seq2seq/raw/d876a1dcacd7aeeeeeaff2c9b806d23116df048f/assets/seq2seq6.png)

이 모델의 Decoder는 다음의 3가지 부분으로 이루어져있습니다.  
* Word Embedding (English)
* Unidirectional GRU
* Fully Connected Layer

Decoder의 경우 encoder와 달리, RNN에 이전 state $s_{t - 1}$와 더불어 attention을 적용한 encoder의 출력값 $a_t^T H$을 함께 입력해주어야 합니다. 또한 이 모델에서는 최종 출력값을 계산할 때 위 그림과 같이 word embedding과 rnn output, attentioned을 적용한 encoder output을 모두 이용한다는 점을 참조해주세요.

각 레이어의 형태를 아래 코드와 같이 정의할 때, decoder module의 forward 함수를 완성해주세요.

In [ ]:
"""TODO
Decoder Module의 forward 함수를 완성해주세요.
"""
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, encoder_dim, decoder_dim,
                 dropout=0.5):
        super().__init__()
        self.embed = nn.Sequential(
            nn.Embedding(output_dim, embed_dim),
            nn.Dropout(dropout)
        )
        self.rnn = nn.GRU(encoder_dim * 2 + embed_dim, decoder_dim)
        self.fc = nn.Linear(
            encoder_dim * 2 + embed_dim + decoder_dim,
            output_dim
        )

    def forward(self, trg, hidden, attended_encoder_outputs):
        embed = self.embed(trg.unsqueeze(0))

        # TODO: embedding과 attended_outputs를 합쳐서 (concatenate) rnn에 전달해줍니다.
        output, hidden = self.rnn(None, None)

        # 모두 길이가 1이므로 0번 dimension을 제거해줍니다.
        embed = embed.squeeze(0)
        output = output.squeeze(0)
        hidden = hidden.squeeze(0)
        attended = attended_encoder_outputs.squeeze(0)
        
        # TODO: embed, output, attended를 모두 이용해 최종 출력값을 계산합니다.
        output_cat = None
        prediction = self.fc(output_cat)

        return prediction, hidden


"""
앞서 작성한 module들을 합쳐 Seq2Seq module을 정의합니다.
"""
import random

class Seq2Seq(nn.Module):
    def __init__(self, input_dim, output_dim, embed_dim,
                 encoder_dim, decoder_dim, dropout=0.5, pad_idx=1):
        super().__init__()
        self.encoder = Encoder(input_dim, embed_dim, encoder_dim, decoder_dim,
                               dropout=dropout)
        self.attention = Attention(encoder_dim, decoder_dim)
        self.decoder = Decoder(output_dim, embed_dim, encoder_dim, decoder_dim,
                               dropout=dropout)
        self.pad_idx = pad_idx

    def forward(self, src, src_len, trg, teacher_force=0.5):
        encoder_outputs, hidden = self.encoder(src, src_len)
        mask = (src != self.pad_idx).T
        a = self.attention(hidden, encoder_outputs, mask)
        attended = (a.T.unsqueeze(-1) * encoder_outputs).sum(0, keepdims=True)
        
        # 각 시점에서 output의 확률을 저장할 placeholder를 생성합니다.
        outputs = torch.zeros(*trg.shape, len(TRG.vocab))
        input = trg[0]
        for t in range(1, trg.size(0)):
            output, hidden = self.decoder(input, hidden, attended)
            outputs[t] = output

            # Teacher forcing을 일정 확률로 적용합니다.
            if random.random() < teacher_force:
                input = trg[t]
            else:
                input = output.argmax(1)
        
        return outputs


# Tests
pad_idx = SRC.vocab.stoi[SRC.pad_token]
seq2seq = Seq2Seq(len(SRC.vocab), len(TRG.vocab), 2, 3, 4, pad_idx=pad_idx)
for _ in range(3):
    sent_pair = next(iter(train_iterator))
    src, src_len = sent_pair.korean
    trg = sent_pair.english
    outputs = seq2seq(src, src_len, trg)
    assert not (outputs[1:] == 0).any()
print("All tests passed")

## Training the Model

In [ ]:
"""
CrossEntropyLoss를 이용해 모델을 학습시킵니다.
Validation dataset을 정의하지 않았으므로 따로 evaluation은 하지 않습니다.
"""
import time
import math

src_pad_idx = SRC.vocab.stoi[SRC.pad_token]
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]

def train(model, iterator, epochs=20, lr=1e-3, teacher_force=0.5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)
    tmp = "Epoch: {:3d} | Time: {:.4f} ms | Loss: {:.4f} | PPL: {:.4f}"

    for epoch in range(epochs):
        start_time = time.time()
        
        avg_loss = 0
        avg_bleu_score = 0
        for batch in iterator:
            src, src_len = batch.korean
            trg = batch.english

            # Loss를 계산합니다.
            model.train()
            outputs = model(src, src_len, trg, teacher_force=teacher_force)
            outputs = outputs[1:].view(-1, outputs.size(-1))
            loss = criterion(outputs, trg[1:].view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            avg_loss += loss.item()
        
        avg_loss /= len(iterator)
        elapsed = time.time() - start_time
        print(tmp.format(epoch + 1, elapsed, avg_loss, math.exp(avg_loss)))

In [ ]:
# 모델을 학습시켜봅시다.
model = Seq2Seq(len(SRC.vocab), len(TRG.vocab), 32, 64, 64,
                pad_idx=src_pad_idx)
train(model, train_iterator)